In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
import streamlit as st
from datetime import datetime
import os
from operator import itemgetter, attrgetter
import numpy as np

import streamlit as st
import plotly.express as px



import warnings    
warnings.filterwarnings("ignore")

/Users/familialima/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#######################################################################################################
                            ######### Carga do Dataframe ##########
#######################################################################################################
file = '/Outputs/MinhasCotacoes.xlsx'
caminho_atual = os.getcwd()
file = caminho_atual + file

minhasacoes = pd.read_excel(file, sheet_name='Cotação das Ações')
minhasacoes

,SETORECONOMICO,SUBSETOR,SEGMENTO,EMPRESA,TITULO,ACAO,QTDECOTAS,VALORCOMPRA,DATACOMPRA,DATACOTACAO,VALORABERTURA,MAIORCOTACAO,MENORCOTACAO,VALORFECHAMENTO,VALORAJUSTADO,DATAPROCESSAMENTO,VALORDIVIDENDO,FLAG_DIVIDENDOS,ANOCOTACAO,DIVIDENDOSANO
0,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-06,17.60,17.75,17.49,17.64,14.48,2024-06-10,0.0,Não,2019,Sim
1,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-07,17.70,17.78,17.58,17.61,14.46,2024-06-10,0.0,Não,2019,Sim
2,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-10,17.61,17.90,17.60,17.65,14.49,2024-06-10,0.0,Não,2019,Sim
3,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-11,17.68,17.95,17.59,17.72,14.55,2024-06-10,0.0,Não,2019,Sim
4,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2019-06-12,17.72,17.79,17.43,17.68,14.51,2024-06-10,0.0,Não,2019,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46002,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-03,7.03,7.12,6.88,6.98,6.98,2024-06-10,0.0,Não,2024,Não
46003,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-04,6.97,6.98,6.35,6.35,6.35,2024-06-10,0.0,Não,2024,Não
46004,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-05,6.35,6.48,6.18,6.27,6.27,2024-06-10,0.0,Não,2024,Não
46005,Consumo Cíclico,Comércio Varejista,Eletrodomésticos,CASAS BAHIA,VIAVAREJO ONNM,BHIA3,249,10.23,2020-03-19,2024-06-06,6.27,6.79,6.27,6.68,6.68,2024-06-10,0.0,Não,2024,Não


In [3]:
minhasacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46007 entries, 0 to 46006
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SETORECONOMICO     46007 non-null  object        
 1   SUBSETOR           46007 non-null  object        
 2   SEGMENTO           46007 non-null  object        
 3   EMPRESA            46007 non-null  object        
 4   TITULO             46007 non-null  object        
 5   ACAO               46007 non-null  object        
 6   QTDECOTAS          46007 non-null  int64         
 7   VALORCOMPRA        46007 non-null  float64       
 8   DATACOMPRA         46007 non-null  datetime64[ns]
 9   DATACOTACAO        46007 non-null  datetime64[ns]
 10  VALORABERTURA      46007 non-null  float64       
 11  MAIORCOTACAO       46007 non-null  float64       
 12  MENORCOTACAO       46007 non-null  float64       
 13  VALORFECHAMENTO    46007 non-null  float64       
 14  VALORA

In [5]:
tkt = 'ABEV3'
df = minhasacoes[(minhasacoes['ACAO'] == tkt)]
vlpago = float(str(minhasacoes['VALORCOMPRA'][minhasacoes['ACAO'] == tkt].iloc[0]))
df['VALORCOMPRA'] = float(vlpago)
df = df.rename(columns={'VALORAJUSTADO':'Cotação no Período','MAIORCOTACAO':'Máxima Cotação Período', 'MENORCOTACAO':'Menor Cotação Período', 'VALORCOMPRA': 'Valor de Compra'})


In [11]:
ult_atualizacao =  str(df['DATACOTACAO'].max()).replace(' 00:00:00','') #Data da ultima att
ult_cotacao = float(str(df.loc[df.index.max(), "Cotação no Período"])) #Pega a última cotacao
menor_cotacao = round(df["Cotação no Período"].min(), 2) #Pega a menor cotacao do periodo
maior_cotacao = round(df["Cotação no Período"].max(), 2) #Pega a maior cotacao do periodo
prim_cotacao = df.loc[df.index.min(), "Cotação no Período"] #Pega a primeira cotacao encontrada
delta = round(((ult_cotacao - vlpago)/ vlpago)*100,2) #A variacao da cotacao no periodo

In [10]:
type(vlpago)

float

In [6]:
# função para selecionar a quantidade de linhas do dataframe
def mostra_qntd_linhas(dataframe):

    qntd_linhas = st.sidebar.slider('Selecione a quantidade de linhas que deseja mostrar na tabela', min_value = 1, max_value = len(dataframe), step = 1)

    #st.write(dataframe.head(qntd_linhas).style.format(subset = ['Valor'], formatter="{:.2f}"))

In [7]:
def retorna_ultima_cotacao_acao(df):
    latest_quotes = df.loc[df.groupby('ACAO')['DATACOTACAO'].idxmax()]
    latest_quotes = latest_quotes[['ACAO', 'DATACOTACAO', 'QTDECOTAS', 'VALORFECHAMENTO']].drop_duplicates()
    latest_quotes['VALORTOTALATUAL'] = latest_quotes['QTDECOTAS'] * latest_quotes['VALORFECHAMENTO']
    return latest_quotes

# Função para retornar um dataframe geral agrupado
def retorna_agrupamento_geral():
  df = minhasacoes[['SETORECONOMICO', 'SUBSETOR', 'SEGMENTO','EMPRESA', 'ACAO','QTDECOTAS', 'VALORCOMPRA']].drop_duplicates()
  df['VALORTOTALCOMPRA'] = df['QTDECOTAS'] * df['VALORCOMPRA']
  df = df.groupby(['SETORECONOMICO', 'SUBSETOR', 'SEGMENTO','EMPRESA', 'ACAO']).agg({
      'QTDECOTAS': np.sum,
      'VALORTOTALCOMPRA': np.sum
  }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
  return df

# Função para retornar um dataframe agrupado
def create_dataframe(coluna):
    df = minhasacoes[[coluna, 'QTDECOTAS', 'VALORCOMPRA', 'ACAO']].drop_duplicates()
    df['VALORTOTALCOMPRA'] = df['QTDECOTAS'] * df['VALORCOMPRA']
    df = df.groupby([coluna, 'ACAO']).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

# Função para consolidar o dataframe
def consolidate_details(df, latest_quotes):
    df = pd.merge(df, latest_quotes[['ACAO', 'DATACOTACAO', 'VALORTOTALATUAL']], on='ACAO', how='inner', validate="many_to_many")
    df = df.drop_duplicates().sort_values(by='ACAO').reset_index(drop=True)
    return df

# Função para agrupar o dataframe
def group_by(df,coluna):
    df = df.groupby(coluna).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum,
        'VALORTOTALATUAL': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

# Função para agrupar o dataframe
def geral_group_by(df):
    df = df.groupby(['SETORECONOMICO', 'SUBSETOR', 'SEGMENTO','EMPRESA', 'ACAO']).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum,
        'VALORTOTALATUAL': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

# Função para atualizar percentual de rendimento e prejuizo
def update_performance(df):
    df['RENDIMENTO(%)'] = ((df['VALORTOTALATUAL'] / df['VALORTOTALCOMPRA'] - 1) * 100).round(2)
    df['PREJUIZO(%)'] = ((df['VALORTOTALCOMPRA'] / df['VALORTOTALATUAL'] - 1) * 100).round(2)
    df['RETORNO(%)'] = ((df['VALORTOTALATUAL'] / df['VALORTOTALCOMPRA'] - 1) * 100).round(2)
    df.loc[df['VALORTOTALATUAL'] < df['VALORTOTALCOMPRA'], 'RENDIMENTO(%)'] = 0
    df.loc[df['VALORTOTALATUAL'] >= df['VALORTOTALCOMPRA'], 'PREJUIZO(%)'] = 0
    return df

# Função que centraliza as funções 
def agupamento(agrupamento, df):
    coluna = agrupamento
    ult_cot = retorna_ultima_cotacao_acao(df)
    dtf = create_dataframe(coluna)
    dtf = consolidate_details(dtf, ult_cot)
    dtf = group_by(dtf, coluna)
    dtf = update_performance(dtf)
    return dtf

# Função que centraliza as funções 
def agupamento_geral(df):
    ult_cot = retorna_ultima_cotacao_acao(df)
    dtf = retorna_agrupamento_geral()
    dtf = consolidate_details(dtf, ult_cot)
    dtf = geral_group_by(dtf)
    dtf = update_performance(dtf)
    return dtf




In [13]:
%pip install streamlit-modal

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
